Importing Kaggle Dataset

In [ ]:
from kagglehub import dataset_download
#remove warnings
import warnings
warnings.filterwarnings("ignore")

from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, TimestampType, LongType
import re

path: str = dataset_download("jinquan/cc-sample-data")

print(path)

spark = SparkSession.builder.appName("payNet").getOrCreate()


Load JSON and clean up Data

In [ ]:
# Read the JSON data from the file
df = spark.read.json(path)

def clean_json_string(json_str):
    """
    Clean JSON string by:
    1. Removing all backslashes
    2. Removing quotes around JSON objects (e.g., "{ }" becomes { })
    """
    if json_str is None:
        return None
    
    # Remove all backslashes
    cleaned = json_str.replace("\\", "")
    
    # Remove quotes around JSON objects - pattern: "{ ... }"
    # This regex finds quoted JSON objects and removes the outer quotes
    cleaned = re.sub(r'"\s*\{\s*(.*?)\s*\}\s*"', r'{\1}', cleaned)
    
    return cleaned

# Register UDF
clean_json_udf = sf.udf(clean_json_string, StringType())

# Apply cleaning to the personal_detail column
df = df.withColumn("personal_detail", clean_json_udf(sf.col("personal_detail")))

# Define schema for address (nested within personal_detail) - all strings initially
address_schema = StructType([
    StructField("street", StringType(), True),
    StructField("city", StringType(), True),
    StructField("state", StringType(), True),
    StructField("zip", StringType(), True)
])

# Define schema for personal_detail - all strings initially
personal_schema = StructType([
    StructField("person_name", StringType(), True),
    StructField("gender", StringType(), True),
    StructField("address", address_schema, True),
    StructField("lat", StringType(), True),
    StructField("long", StringType(), True),
    StructField("city_pop", StringType(), True),
    StructField("job", StringType(), True),
    StructField("dob", StringType(), True)
])

# Parse the cleaned JSON string into proper columns (overwrite the original column)
df_with_parsed_personal = df.withColumn("personal_detail", sf.from_json(sf.col("personal_detail"), personal_schema))

# for debugging
# df_with_parsed_personal.select(col("personal_detail.person_name")).show()
# df_with_parsed_personal.select(col("personal_detail.address.street")).show()

# Process name splitting for 'first' and 'last' names with improved robustness.
df_with_names = df_with_parsed_personal.withColumn(
    "cleaned_person_name",
    sf.when(sf.col("personal_detail.person_name").isNotNull(),
        sf.trim(
            sf.regexp_replace( # Normalize multiple spaces to single space
                sf.regexp_replace( # Replace all non-alphanumeric characters (except spaces) with a single space
                    sf.regexp_replace( # Remove specific trailing strings like 'eeeee' and 'N' followed by 4 or more '0' or 'O' (case-insensitive)
                        sf.regexp_replace(sf.col("personal_detail.person_name"), r"(?i),?eeeee$", ""),
                        r"(?i),?\s*N[0O]{4,}$", "" # Updated regex to handle N0000, NOOOO etc.
                    ),
                    r"[^a-zA-Z0-9\s]", " " # Replace any character that is NOT a letter, number, or whitespace with a space. This will catch /, !, @, |, and also the comma if it's not part of a "Lastname, Firstname" pattern.
                ),
                r"\s+", " " # Normalize multiple spaces to single space
            )
        )
    ).otherwise(None)
)

df_with_names = df_with_names \
    .withColumn("name_parts", sf.split(sf.col("cleaned_person_name"), " ")) \
    .withColumn("first", 
        sf.when(sf.size(sf.col("name_parts")) >= 1, sf.trim(sf.element_at(sf.col("name_parts"), 1)))
        .otherwise(None)
    ) \
    .withColumn("last", 
        sf.when(sf.size(sf.col("name_parts")) > 1, 
                sf.trim(sf.concat_ws(" ", sf.slice(sf.col("name_parts"), 2, sf.size(sf.col("name_parts"))))))
        .otherwise(None)
    ) \
    .drop("cleaned_person_name", "name_parts") # Drop intermediate columns


# Flatten the personal_detail structure and address structure
df_flattened = df_with_names.select(
    # Original columns in desired order
    sf.col("Unnamed: 0"),
    sf.col("trans_date_trans_time"),
    sf.col("cc_num"),
    sf.col("merchant"),
    sf.col("category"),
    sf.col("amt"),
    
    sf.col("first"),
    sf.col("last"),

    # Personal details
    sf.col("personal_detail.gender").alias("gender"),
    
    # Flattened address details
    sf.col("personal_detail.address.street").alias("street"),
    sf.col("personal_detail.address.city").alias("city"),
    sf.col("personal_detail.address.state").alias("state"),
    sf.col("personal_detail.address.zip").alias("zip"),
    
    # Location and demographic info
    sf.col("personal_detail.lat").alias("lat"),
    sf.col("personal_detail.long").alias("long"),
    sf.col("personal_detail.city_pop").alias("city_pop"),
    sf.col("personal_detail.job").alias("job"),
    sf.col("personal_detail.dob").alias("dob"),
    
    # Transaction details
    sf.col("trans_num"),
    sf.col("merch_lat"),
    sf.col("merch_long"),
    sf.col("is_fraud"),
    sf.col("merch_zipcode"),
    sf.col("merch_last_update_time"),
    sf.col("merch_eff_time"),
    sf.col("cc_bic")
)


# Type conversions and rounding in one operation, including date format and timezone handling. Assuming initial timzeone is UTC and converting to UTC
df_cleaned = df_flattened.withColumns({
    'Unnamed: 0': sf.col("Unnamed: 0").cast(IntegerType()),


    # Convert trans_date_trans_time to TimestampType, then to UTC+8, then format
    'trans_date_trans_time': sf.date_format(sf.from_utc_timestamp(sf.col("trans_date_trans_time").cast("timestamp"), "UTC+8"), 'yyyy-MM-dd HH:mm:ss Z'),


    'amt': sf.round(sf.col("amt").cast(FloatType()), 6),
    'merch_lat': sf.round(sf.col("merch_lat").cast(FloatType()), 6),
    'merch_long': sf.round(sf.col("merch_long").cast(FloatType()), 6),
    'is_fraud': sf.col("is_fraud").cast(IntegerType()),


    # Convert merch_eff_time (microseconds) to TimestampType, then to UTC+8, then format
 
    'merch_eff_time': sf.date_format(
        sf.from_utc_timestamp(
            (
                sf.rpad(
                    sf.col("merch_eff_time").cast(StringType()),
                    16,
                    '0'
                ).cast(LongType()) / 1000000
            ).cast("timestamp"),
            "UTC+8"
        ),
        'yyyy-MM-dd HH:mm:ss.SSSSSS Z'
    ),


    # Convert merch_last_update_time (microseconds) to TimestampType, then to UTC+8, then format
    'merch_last_update_time': sf.date_format(
        sf.from_utc_timestamp(
            (
                sf.rpad(
                    sf.col("merch_last_update_time").cast(StringType()),
                    16,
                    '0'
                ).cast(LongType()) / 1000000
            ).cast("timestamp"),
            "UTC+8"
            ),
            'yyyy-MM-dd HH:mm:ss.SSSSSS Z'
        ), 
    
    'lat': sf.round(sf.col("lat").cast(FloatType()), 6),
    'long': sf.round(sf.col("long").cast(FloatType()), 6),
    'city_pop': sf.col("city_pop").cast(IntegerType())
})


# Handle null values and "NA" strings for all string columns automatically
string_columns = [field.name for field in df_cleaned.schema.fields if field.dataType.typeName() == 'string']

# Create dictionary for null value handling across all string columns
null_handling_dict = {}
for col_name in string_columns:
    null_handling_dict[col_name] = sf.when(
        (sf.lower(sf.col(col_name)) == "na") | 
        (sf.lower(sf.col(col_name)) == "null") | 
        (sf.col(col_name) == ""), 
        None
    ).otherwise(sf.col(col_name))

df_cleaned = df_cleaned.withColumns(null_handling_dict)

#all merchant names start with the word 'fraud_'; safe to remove
df_cleaned = df_cleaned.withColumn(
    "merchant",
    sf.regexp_replace(sf.col("merchant"), "fraud_", "")
)

## Display cleaned data 

# Show final result
df_cleaned.show(40,truncate=False)

#convert subsection to markdown for viewing
# print(df_cleaned.filter(sf.col("Unnamed: 0") <= 25).toPandas().to_markdown())

# Show schema to verify structure
df_cleaned.printSchema()

Handling PII data

In [ ]:
# Direct Identifiers
# cc_num, first, last, trans_num, dob

# Indirect Identifiers
# city, state, zip, street, job, lat, long, cc_bic

# Indentifiers to delete : 
identifier_delete = [
    # not needed for analysis
    "cc_num",
    "first",
    "last",
    "trans_num",
    "cc_bic",
    # specific location not required
    "street",
]

# reducing specificity of location by rounding latitude and longitude
df_cleaned = df_cleaned.withColumn("lat", sf.round(sf.col("lat"), 2))
df_cleaned = df_cleaned.withColumn("long", sf.round(sf.col("long"), 2))

# remove columns 
df_cleaned = df_cleaned.drop(*identifier_delete)

# keep only the year (first 4 digits) from dob
df_cleaned = df_cleaned.withColumn("dob", sf.substring(sf.col("dob"), 1, 4))

df_cleaned.show(40,truncate=False)
print(df_cleaned.tail(40))

df_cleaned.printSchema()

Interactive Map

In [34]:
import os
import pandas as pd
import dash
from dash import html
from dash import dcc
import plotly.express as px
import folium
from folium.plugins import HeatMap

pandas_df = df_cleaned.select(["lat", "long", "amt"]).toPandas()

print(f'Pandas dataframe shape: {pandas_df.shape}')

heat_data = pandas_df.to_numpy().tolist()

if not pandas_df.empty:
    map_center = [pandas_df.lat.mean(), pandas_df.long.mean()]
else:
    map_center = [0, 0]

# Set the desired initial zoom level for the Folium map
desired_zoom_level = 7
m = folium.Map(location=map_center, zoom_start=desired_zoom_level, tiles="OpenStreetMap")

HeatMap(heat_data).add_to(m)

assets_folder = 'assets'
if not os.path.exists(assets_folder):
    os.makedirs(assets_folder)
folium_map_path = os.path.join(assets_folder, 'folium_heatmap.html')
m.save(folium_map_path)
print(f"\nFolium map saved to {folium_map_path}")

# Initialize Dash for inline display in notebooks (Dash 2.11+ compatible)
app = dash.Dash(__name__) 

# Define the Dash app layout
app.layout = html.Div(
    style={'fontFamily': 'Arial, sans-serif', 'padding': '20px', 'backgroundColor': '#f0f2f5'},
    children=[
        html.H1("Data Dashboard",
                style={'textAlign': 'center', 'color': '#333', 'marginBottom': '30px'}),

        html.Div(
            style={'display': 'flex', 'flexDirection': 'row', 'gap': '20px'},
            children=[
                # Left panel for the Folium Map
                html.Div(
                    style={'display':'flex','flexDirection': 'column','justify_content': 'center','align_items': 'center','flex': '1', 'backgroundColor': 'white', 'padding': '20px', 'borderRadius': '8px', 'boxShadow': '0 2px 4px rgba(0,0,0,0.1)'},
                    children=[
                        html.H2("Expenditure Heatmap", style={'textAlign': 'center', 'color': '#555'}),
                        # Use an Iframe to embed the Folium HTML map
                        html.Iframe(
                            id='folium-map-iframe',
                            srcDoc=open(folium_map_path, 'r').read(), # Read the HTML content directly
                            style={'width': '100%','height': '400px', 'border': 'none', 'borderRadius': '4px'}
                        )
                    ]
                ),

            ]
        )
    ]
)


print("\nRunning Dash application inline in Jupyter Notebook")
app.run(mode='inline', port=8050) # Using mode='inline' for Jupyter Notebook display

Pandas dataframe shape: (1296675, 3)

Folium map saved to assets/folium_heatmap.html

Running Dash application inline in Jupyter Notebook
